In [1]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

import random

#

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, KFold

from sklearn.metrics import confusion_matrix

#

from bayes_opt import BayesianOptimization

In [152]:
dfs = []

for n in range( 100 ):

    df = pd.DataFrame()

    df[ 'x0' ] = [ random.randint( 0, 100 ) for i in range( 1000 ) ]

    df[ 'x1' ] = [ random.randint( 0, 100 ) for i in range( 1000 ) ]

    df[ 'x2' ] = [ random.randint( 0, 100 ) for i in range( 1000 ) ]

    df[ 'y' ] = [ round( np.sin( np.sqrt( x0 ** 2 + x1 ** 3 + x2 ** 4 ) ) ** 2 ) for x0, x1, x2 in zip( df[ 'x0' ], df[ 'x1' ], df[ 'x2' ] ) ]

    dfs.append( df )

train_dfs, test_dfs = dfs[ : 80 ], dfs[ 80 : ]

#

X_colnames = [ 'x0', 'x1', 'x2' ]

y_colname = 'y'

In [164]:
def function_to_maximize( max_depth, min_samples_split, min_samples_leaf ):

    value = cfs.BinaryClassification_DecisionTree_CV( train_dfs = train_dfs, \
                                                      X_colnames = X_colnames, \
                                                      y_colname = y_colname, \
                                                      metric_function = cfs.mf_balanced_accuracy, \
                                                      dt_random_state = 1, \
                                                      cv_num_of_splits = 10, \
                                                      cv_shuffle = False, \
                                                      cv_random_state = None, \
                                                      y_labels = None, \
                                                      criterion = 'entropy', \
                                                      max_depth = round( max_depth ),
                                                      min_samples_split = round( min_samples_split ),
                                                      min_samples_leaf = round( min_samples_leaf ) )

    return value

In [165]:
parameter_bounds = {
    
    'max_depth': ( 1, 100 ),

    'min_samples_split': ( 2, 100 ),

    'min_samples_leaf': ( 1, 100 )
    
}

optimizer = BayesianOptimization( f = function_to_maximize, \
                                  pbounds = parameter_bounds, \
                                  random_state = 0, \
                                  verbose = 1 )

optimizer.maximize( n_iter = 25, init_points = 5 ) # Bayesian optimization

best_params_bayes = optimizer.max[ 'params' ]

best_score_bayes = optimizer.max['target']

print( f'Tuned Hyperparameters: { best_params_bayes }' )

print( f'Best Score: { best_score_bayes }' )

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 2         | 0.6029    | 54.94     | 42.94     | 65.3      |
| 6         | 0.6152    | 58.75     | 30.88     | 69.75     |
| 7         | 0.6319    | 65.08     | 2.48      | 74.12     |
| 10        | 0.6586    | 95.86     | 1.956     | 32.98     |
| 12        | 0.7178    | 72.56     | 1.067     | 6.21      |
| 14        | 0.7486    | 86.7      | 1.37      | 2.01      |
Tuned Hyperparameters: {'max_depth': np.float64(86.69907027072173), 'min_samples_leaf': np.float64(1.3704453924896907), 'min_samples_split': np.float64(2.009616539983294)}
Best Score: 0.7486033295298407


In [166]:
for key, val in best_params_bayes.items():

    best_params_bayes[ key ] = round( val )

tn, fp, fn, tp = cfs.BinaryClassification_DecisionTree( train_dfs = train_dfs, \
                                                        test_dfs = test_dfs, \
                                                        X_colnames = X_colnames, \
                                                        y_colname = y_colname, \
                                                        dt_random_state = 0, \
                                                        y_labels = None, \
                                                        **best_params_bayes ).values()

cfs.mf_balanced_accuracy( tn = tn, fp = fp, fn = fn, tp = tp )

np.float64(0.7648223851267582)

In [148]:
##### Initial Experimentation Below

In [101]:
def BinaryClassification_DecisionTree_CV( X_train, \
                                          y_train, \
                                          metric_function, \
                                          dt_random_state = 0, \
                                          cv_num_of_splits = 10, \
                                          cv_shuffle = False, \
                                          y_labels = None, \
                                          **dt_hyperparameters ):

    #

    model = DecisionTreeClassifier( random_state = dt_random_state, **dt_hyperparameters )

    #

    kfold = KFold( n_splits = cv_num_of_splits, shuffle = cv_shuffle )

    #

    validation_test_metric_value_list = []

    for fold in kfold.split( X_train ):

        validation_train_indexes, validation_test_indexes = fold

        #

        X_validation_train, X_validation_test = X_train[ validation_train_indexes ], X_train[ validation_test_indexes ]

        y_validation_train, y_validation_test = y_train[ validation_train_indexes ], y_train[ validation_test_indexes ]

        #

        model = model.fit( X_validation_train, y_validation_train )

        #

        y_pred_validation_test = model.predict( X_validation_test )

        #

        tn, fp, fn, tp = confusion_matrix( y_validation_test, y_pred_validation_test, labels = y_labels ).ravel()

        #

        validation_test_metric_value = metric_function( tn, fp, fn, tp )

        validation_test_metric_value_list.append( validation_test_metric_value )

    #

    mean_validation_test_metric_value = np.mean( validation_test_metric_value_list )

    #

    return mean_validation_test_metric_value

In [102]:
def accuracy( tn, fp, fn, tp, **kwargs ):

    return ( tp + tn ) / ( tp + tn + fp + fn )

In [105]:
X = np.array( [ [ random.randint( -5, 5 ) for i in range( 5 ) ] for i in range( 100 ) ] )

y = np.array( [ random.randint( 0, 1 ) for i in range( 100 ) ] )

#

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2 )

In [109]:
parameters = {

    'criterion': 'entropy', 
    
    'max_depth': 1,
    
    'min_samples_split': 9,
    
    'min_samples_leaf': 10,
    
}

BinaryClassification_DecisionTree_CV( X_train, \
                                      y_train, \
                                      metric_function = accuracy, \
                                      **parameters )

np.float64(0.625)

In [107]:
def function_to_maximize( max_depth, min_samples_split, min_samples_leaf ):

    value = BinaryClassification_DecisionTree_CV( X_train = X_train, \
                                                  y_train = y_train, \
                                                  metric_function = accuracy, \
                                                  dt_random_state = 0, \
                                                  cv_num_of_splits = 10, \
                                                  cv_shuffle = False, \
                                                  y_labels = None, \
                                                  criterion = 'entropy', \
                                                  max_depth = round( max_depth ), \
                                                  min_samples_split = round( min_samples_split ), \
                                                  min_samples_leaf = round( min_samples_leaf ) )

    return value

In [108]:
parameter_bounds = {
    
    'max_depth': ( 1, 10 ),
    
    'min_samples_split': ( 2, 10 ),
    
    'min_samples_leaf': ( 1, 10 )
    
}

optimizer = BayesianOptimization( f = function_to_maximize, 
                                  pbounds = parameter_bounds, 
                                  random_state = 0 )

optimizer.maximize( n_iter = 25, init_points = 10 ) # Bayesian optimization

best_params_bayes = optimizer.max[ 'params' ]

best_params_bayes['max_depth'] = round(best_params_bayes['max_depth'])
best_params_bayes['min_samples_split'] = round(best_params_bayes['min_samples_split'])
best_params_bayes['min_samples_leaf'] = round(best_params_bayes['min_samples_leaf'])

best_score_bayes = optimizer.max['target']

print( best_params_bayes, best_score_bayes )

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | 0.5625    | 5.939     | 7.437     | 6.822     |
| 2         | 0.575     | 5.904     | 4.813     | 7.167     |
| 3         | 0.5625    | 4.938     | 9.026     | 9.709     |
| 4         | 0.4875    | 4.451     | 8.126     | 6.231     |
| 5         | 0.5625    | 6.112     | 9.33      | 2.568     |
| 6         | 0.575     | 1.784     | 1.182     | 8.661     |
| 7         | 0.5625    | 8.003     | 8.83      | 9.829     |
| 8         | 0.5625    | 8.192     | 5.153     | 8.244     |
| 9         | 0.5875    | 2.064     | 6.759     | 3.147     |
| 10        | 0.55      | 9.502     | 5.697     | 5.317     |
| 11        | 0.5625    | 7.023     | 7.393     | 8.771     |
| 12        | 0.4625    | 3.073     | 9.618     | 5.066     |
| 13        | 0.6125    | 1.298     | 4.881     | 2.429     |
| 14        | 0.5       | 3.33      | 4.269     | 2.341     |
| 15    

In [161]:
def BinaryClassification_DecisionTree_CV( train_dfs, \
                                          X_colnames, \
                                          y_colname, \
                                          metric_function, \
                                          dt_random_state = 0, \
                                          cv_num_of_splits = 10, \
                                          cv_shuffle = False, \
                                          cv_random_state = None, \
                                          y_labels = None, \
                                          **dt_hyperparameters ):

    #

    train_dfs_indexes = np.array( [ n for n in range( len( train_dfs ) ) ] )

    #

    kfold = KFold( n_splits = cv_num_of_splits, shuffle = cv_shuffle, random_state = cv_random_state )

    #

    validation_test_metric_value_list = []

    for fold in kfold.split( train_dfs_indexes ):

        validation_train_indexes, validation_test_indexes = fold

        #

        validation_train_dfs = [ df for index, df in enumerate( train_dfs ) if index in validation_train_indexes ]

        validation_test_dfs = [ df for index, df in enumerate( train_dfs ) if index in validation_test_indexes ]

        #

        tn, fp, fn, tp = BinaryClassification_DecisionTree( train_dfs = validation_train_dfs, \
                                                            test_dfs = validation_test_dfs, \
                                                            X_colnames = X_colnames, \
                                                            y_colname = y_colname, \
                                                            dt_random_state = dt_random_state, \
                                                            y_labels = y_labels, \
                                                            **dt_hyperparameters )

        #

        validation_test_metric_value = metric_function( tn, fp, fn, tp )

        validation_test_metric_value_list.append( validation_test_metric_value )

    #

    mean_validation_test_metric_value = np.mean( validation_test_metric_value_list )

    #

    return mean_validation_test_metric_value

In [210]:
dfs = []

for n in range( 100 ):

    df = pd.DataFrame()

    df[ 'x0' ] = [ random.randint( 0, 100 ) for i in range( 20 ) ]

    df[ 'y' ] = [ n % 2 for n in df[ 'x0' ] ]

    dfs.append( df )

train_dfs, test_dfs = dfs[ : 80 ], dfs[ 80 : ]

In [215]:
def function_to_maximize( max_depth, min_samples_split, min_samples_leaf ):

    value = BinaryClassification_DecisionTree_CV_dfver( train_dfs = train_dfs, \
                                                X_colnames = [ 'x0' ], \
                                                y_colname = 'y', \
                                                metric_function = accuracy, \
                                                dt_random_state = 0, \
                                                cv_num_of_splits = 10, \
                                                cv_shuffle = False, \
                                                cv_random_state = None, \
                                                y_labels = [ 0, 1 ], \
                                                criterion = 'entropy', \
                                                max_depth = round( max_depth ), \
                                                min_samples_split = round( min_samples_split ), \
                                                min_samples_leaf = round( min_samples_leaf ) )

    return value

In [216]:
parameter_bounds = {
    
    'max_depth': ( 1, 1000 ),
    
    'min_samples_split': ( 2, 1000 ),
    
    'min_samples_leaf': ( 1, 1000 )
    
}

optimizer = BayesianOptimization( f = function_to_maximize, 
                                  pbounds = parameter_bounds, 
                                  random_state = 0, verbose = 1 )

optimizer.maximize( n_iter = 100, init_points = 50 ) # Bayesian optimization

best_params_bayes = optimizer.max[ 'params' ]

for key, val in best_params_bayes.items():

    best_params_bayes[ key ] = round( val )

best_score_bayes = optimizer.max['target']

print( best_params_bayes, best_score_bayes )

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 2         | 0.5431    | 545.3     | 424.2     | 646.6     |
| 12        | 0.5531    | 568.9     | 19.77     | 618.4     |
| 53        | 0.655     | 515.4     | 10.83     | 463.6     |
| 54        | 0.7506    | 550.5     | 8.67      | 416.0     |
| 55        | 0.8037    | 554.3     | 7.955     | 370.4     |
| 57        | 0.8138    | 601.7     | 4.468     | 355.6     |
| 94        | 0.8238    | 624.5     | 1.64      | 341.7     |
| 121       | 0.8244    | 630.1     | 3.8       | 331.8     |
| 137       | 0.8275    | 654.2     | 1.048     | 329.3     |
| 150       | 0.8344    | 483.6     | 5.737     | 318.6     |
{'max_depth': 484, 'min_samples_leaf': 6, 'min_samples_split': 319} 0.834375


In [213]:
def BinaryClassification_DecisionTree( train_dfs, \
                                       test_dfs, \
                                       X_colnames, \
                                       y_colname, \
                                       dt_random_state = 0, \
                                       y_labels = None, \
                                       **dt_hyperparameters ):

    #

    model = DecisionTreeClassifier( random_state = dt_random_state, **dt_hyperparameters )

    #

    train_df = pd.concat( train_dfs )

    test_df = pd.concat( test_dfs )

    #

    X_train, X_test = train_df[ X_colnames ], test_df[ X_colnames ]

    y_train, y_test = train_df[ y_colname ], test_df[ y_colname ]

    #

    model = model.fit( X_train, y_train )

    #

    y_pred_test = model.predict( X_test )

    #

    tn, fp, fn, tp = confusion_matrix( y_test, y_pred_test, labels = y_labels ).ravel()

    #

    return tn, fp, fn, tp

In [214]:
tn, fp, fn, tp = BinaryClassification_DecisionTree( train_dfs = train_dfs, \
                                       test_dfs = test_dfs, \
                                       X_colnames = [ 'x0' ], \
                                       y_colname = 'y', \
                                       dt_random_state = 0, \
                                       y_labels = None, \
                                       **best_params_bayes )

accuracy( tn, fp, fn, tp )

np.float64(0.775)